In [2]:
!which aws
!aws --version
!python --version
!pip install scikit-learn==0.23

/usr/local/bin/aws
aws-cli/2.13.28 Python/3.11.6 Darwin/23.0.0 exe/x86_64 prompt/off
Python 3.8.13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 8.8 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deephyper 0.4.2 requires scikit-learn>=0.23.1, but you have scikit-learn 0.23.0 which is incompatible.
pyldavis 3.4.0 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.0 which is incompatible.


In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
import pickle
california_housing = fetch_california_housing(as_frame=True)

In [1]:
import sklearn
sklearn.__version__

'1.3.2'

In [3]:
features_of_interest = ["AveRooms", "HouseAge", "MedInc"]

In [8]:
df = california_housing.frame
df.corr()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
MedInc,1.000000,-0.119034,0.326895,-0.062040,0.004834,0.018766,-0.079809,-0.015176,0.688075
HouseAge,-0.119034,1.000000,-0.153277,-0.077747,-0.296244,0.013191,0.011173,-0.108197,0.105623
AveRooms,0.326895,-0.153277,1.000000,0.847621,-0.072213,-0.004852,0.106389,-0.027540,0.151948
AveBedrms,-0.062040,-0.077747,0.847621,1.000000,-0.066197,-0.006181,0.069721,0.013344,-0.046701
Population,0.004834,-0.296244,-0.072213,-0.066197,1.000000,0.069863,-0.108785,0.099773,-0.024650
AveOccup,0.018766,0.013191,-0.004852,-0.006181,0.069863,1.000000,0.002366,0.002476,-0.023737
Latitude,-0.079809,0.011173,0.106389,0.069721,-0.108785,0.002366,1.000000,-0.924664,-0.144160
Longitude,-0.015176,-0.108197,-0.027540,0.013344,0.099773,0.002476,-0.924664,1.000000,-0.045967
MedHouseVal,0.688075,0.105623,0.151948,-0.046701,-0.024650,-0.023737,-0.144160,-0.045967,1.000000


In [10]:
alphas = np.logspace(-2, 1, num=10)
model_RCV = make_pipeline(StandardScaler(), RidgeCV(alphas=alphas))
model_LR = make_pipeline(StandardScaler(), LinearRegression())
model_RCV = RidgeCV(alphas=alphas)

cv_results = cross_validate(
    model_RCV,
    california_housing.data,
    california_housing.target,
    return_estimator=True,
    n_jobs=2,
    return_train_score=True,
    scoring=('r2', 'neg_mean_squared_error', 'neg_root_mean_squared_error'),
)

In [11]:
r2score = cv_results["test_r2"]
MSEscore = cv_results["test_neg_mean_squared_error"]
RMSEscore = cv_results["test_neg_root_mean_squared_error"]
print(f"R2 score: {r2score.mean():.3f} ± {r2score.std():.3f}")
print(f"MSE score: {abs(MSEscore).mean():.3f} ± {MSEscore.std():.3f}")
print(f"RMSE score: {abs(RMSEscore).mean():.3f} ± {RMSEscore.std():.3f}")

R2 score: 0.553 ± 0.062
MSE score: 0.558 ± 0.066
RMSE score: 0.746 ± 0.044


In [12]:
for model in cv_results['estimator']:
    print(model.coef_)

[ 4.36324459e-01  1.09163356e-02 -1.17209404e-01  7.93743716e-01
 -9.89437103e-06 -4.14761383e-03 -4.37243225e-01 -4.75119123e-01]
[ 4.27137746e-01  9.74470589e-03 -8.46175617e-02  5.34704592e-01
 -8.92649450e-06 -3.16323624e-03 -4.16914476e-01 -4.33395578e-01]
[ 4.34229725e-01  1.02616769e-02 -1.10486937e-01  6.24299276e-01
  1.57264658e-05 -3.49231859e-03 -4.26600549e-01 -4.47936751e-01]
[ 4.26309850e-01  6.30712353e-03 -8.58648141e-02  5.34184586e-01
 -1.43225079e-05 -3.33987706e-03 -4.09856693e-01 -3.92975692e-01]
[ 4.46491698e-01  9.43405756e-03 -1.18443795e-01  6.97099028e-01
 -1.37612460e-06 -8.82312352e-03 -4.21454191e-01 -4.29277972e-01]


In [ ]:
model.predict(np.array([[1, 2,3,4,5,6,7,8]]))

In [ ]:
pickle.dump(model, open('cal_model.pkl', 'wb'))